In [ ]:
import os
import wave
import pyaudio
import string
from openai import OpenAI
import speech_recognition as sr

# Initialize recognizer
recognizer = sr.Recognizer()
client = OpenAI(api_key='insertYourOwnAPIKey')  # Change to your API key (or reach out to me for my own)

def r2d2_beep_from_text(text, sounds_folder):
    # Normalize text
    word = ''.join(c for c in text.lower() if c in string.ascii_lowercase)

    # Set up audio stream
    p = pyaudio.PyAudio()
    stream = p.open(format=p.get_format_from_width(2),
                    channels=1,
                    rate=22050,
                    output=True)

    # Prepare full sound data
    data = b""

    for letter in word:
        sound_path = os.path.join(sounds_folder, f"{letter}.wav")
        if not os.path.exists(sound_path):
            print(f"Warning: No sound for letter '{letter}'")
            continue
        try:
            with wave.open(sound_path, "rb") as f:
                frames = f.readframes(f.getnframes())
                data += frames
        except Exception as e:
            print(e)

    # Play concatenated sound
    if data:
        stream.write(data)

    # Cleanup
    stream.stop_stream()
    stream.close()
    p.terminate()

# Listen to user speech
with sr.Microphone() as source:
    print("Listening for your command...")
    recognizer.adjust_for_ambient_noise(source)
    audio = recognizer.listen(source)

try:
    # Convert spoken words to text
    text = recognizer.recognize_google(audio)
    print("You said:", text)

    # Generate GPT-3.5 response
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are R2D2, an astromech droid. You respond in cheerful, curious ways."},
            {"role": "user", "content": text}
        ]
    )
    ai_reply = completion.choices[0].message.content
    print("R2D2 (translated response):", ai_reply)

    # Play R2D2 beeps matching GPT reply
    print("R2D2 is responding with matching astromech beeps...")
    sounds_folder = 'sounds/'  
    r2d2_beep_from_text(ai_reply, sounds_folder)

except sr.UnknownValueError:
    print("Could not understand audio.")
except sr.RequestError as e:
    print(f"Speech Recognition request error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")
